In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

## Implement Teleportation or Superdense Coding Protocol with all four bell state as e-bit

Consider all the bell state as e-bit
Running the  simulator shows that Alice and Bob's classical bits or quantum bit always agree.

In [3]:
qc = QuantumCircuit(2)
qc.h(0)

In [2]:

sim = AerSimulator()

In [3]:

# Bell state preparation functions
def bell_state(state_id):
    """
    0 -> |Φ+> = (|00> + |11>)/√2
    1 -> |Φ-> = (|00> - |11>)/√2
    2 -> |Ψ+> = (|01> + |10>)/√2
    3 -> |Ψ-> = (|01> - |10>)/√2
    """
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)

    if state_id == 1:   # |Φ->
        qc.z(0)
    elif state_id == 2: # |Ψ+>
        qc.x(0)
    elif state_id == 3: # |Ψ->
        qc.x(0)
        qc.z(0)

    return qc

In [6]:
# Corrections needed based on starting Bell state
# These corrections undo the effect of a rotated Bell basis
# and restore proper superdense decoding
bell_corrections = {
    0: (0, 0),  # Φ+: no correction
    1: (0, 1),  # Φ−: flip d
    2: (1, 0),  # Ψ+: flip c
    3: (1, 1)   # Ψ−: flip both
}

In [9]:
def superdense_test(state_id, c, d):
    """
    Universal version that corrects Bob's output so that
    the decoded bits always match the encoded bits.
    """

    A = QuantumRegister(1, "Alice")
    B = QuantumRegister(1, "Bob")
    Cc = ClassicalRegister(1, "c_out")
    Cd = ClassicalRegister(1, "d_out")

    qc = QuantumCircuit(A, B, Cc, Cd)

    # Prepare chosen Bell state
    qc.compose(bell_state(state_id), [A[0], B[0]], inplace=True)
    qc.barrier()

    # Alice encodes (c,d)
    if d == 1:
        qc.z(A[0])
    if c == 1:
        qc.x(A[0])
    qc.barrier()

    # Bob's decoding step
    qc.cx(A[0], B[0])
    qc.h(A[0])
    qc.barrier()

    # Measure
    qc.measure(A[0], Cc)
    qc.measure(B[0], Cd)

    # Run simulator
    result = sim.run(qc).result()
    counts = result.get_counts()

    # Extract Bob’s raw bits
    raw = list(counts.keys())[0].replace(" ", "")
    bob_c = int(raw[0])  # first bit
    bob_d = int(raw[1])  # second bit

    # Apply correction depending on Bell state
    corr_c, corr_d = bell_corrections[state_id]
    corrected_c = bob_c ^ corr_c
    corrected_d = bob_d ^ corr_d

    return (corrected_c, corrected_d), (bob_c, bob_d), qc


In [11]:

# Test all Bell states with all 4 possible messages
bell_names = ["Φ+", "Φ−", "Ψ+", "Ψ−"]
messages = [(0,0), (0,1), (1,0), (1,1)]
for state_id in range(4):
    print(f"\n=== Bell State: |{bell_names[state_id]}> ===")
    for c, d in messages:
        corrected, raw, _ = superdense_test(state_id, c, d)
        print(f"Alice sent {(c,d)} | "
              f"Bob raw {raw} | "
              f"Corrected output {corrected}")


=== Bell State: |Φ+> ===
Alice sent (0, 0) | Bob raw (0, 0) | Corrected output (0, 0)
Alice sent (0, 1) | Bob raw (0, 1) | Corrected output (0, 1)
Alice sent (1, 0) | Bob raw (1, 0) | Corrected output (1, 0)
Alice sent (1, 1) | Bob raw (1, 1) | Corrected output (1, 1)

=== Bell State: |Φ−> ===
Alice sent (0, 0) | Bob raw (0, 1) | Corrected output (0, 0)
Alice sent (0, 1) | Bob raw (0, 0) | Corrected output (0, 1)
Alice sent (1, 0) | Bob raw (1, 1) | Corrected output (1, 0)
Alice sent (1, 1) | Bob raw (1, 0) | Corrected output (1, 1)

=== Bell State: |Ψ+> ===
Alice sent (0, 0) | Bob raw (1, 0) | Corrected output (0, 0)
Alice sent (0, 1) | Bob raw (1, 1) | Corrected output (0, 1)
Alice sent (1, 0) | Bob raw (0, 0) | Corrected output (1, 0)
Alice sent (1, 1) | Bob raw (0, 1) | Corrected output (1, 1)

=== Bell State: |Ψ−> ===
Alice sent (0, 0) | Bob raw (1, 1) | Corrected output (0, 0)
Alice sent (0, 1) | Bob raw (1, 0) | Corrected output (0, 1)
Alice sent (1, 0) | Bob raw (0, 1) | Corre